Übung:  09-2 Gateway API
-------------------

Gateway API ist ein offizielles Kubernetes-Projekt, das sich auf L4- und L7-Routing in Kubernetes konzentriert. Dieses Projekt stellt die nächste Generation von Kubernetes Ingress-, Load Balancing- und Service Mesh-APIs dar. Es war von Anfang an generisch, ausdrucksstark und rollenorientiert konzipiert.

Das Gesamtressourcenmodell konzentriert sich auf drei separate [Personas](https://gateway-api.sigs.k8s.io/concepts/roles-and-personas) (Infrastructur, Cluster, Application) und entsprechende Ressourcen, die von ihnen verwaltet werden sollen:

![](https://gateway-api.sigs.k8s.io/images/resource-model.png)

Quelle: [Gateway API](https://gateway-api.sigs.k8s.io/)
- - -

Um die Gateway-Ressourcen verwenden zu können, müssen diese zunächst erstellt werden, da sie nicht Teil des Standard-Kubernetes sind. 

Anschliessend muss ein Gateway-fähiger Ingress-Controller, hier [projectcontour](https://projectcontour.io/getting-started/), installiert werden. 

Weitere Informationen zur Erstellung der Ressourcen findest man [hier](https://gateway-api.sigs.k8s.io/guides/#installing-a-gateway-controller).

In [ ]:
! kubectl apply -f https://projectcontour.io/quickstart/contour-gateway-provisioner.yaml

Beim Installieren des Cluster haben die Persona **"Infrastructure Provider"** die GatewayClass erstellt:

In [ ]:
%%bash
kubectl apply -f - <<EOF
kind: GatewayClass
apiVersion: gateway.networking.k8s.io/v1
metadata:
  name: contour
spec:
  controllerName: projectcontour.io/gateway-controller
EOF

Und die Persona **"Cluster Operators"** die Gateways:

In [ ]:
%%bash
kubectl apply -f - <<EOF
kind: Gateway
apiVersion: gateway.networking.k8s.io/v1
metadata:
  name: contour
  namespace: projectcontour
spec:
  gatewayClassName: contour
  listeners:
    - name: http
      protocol: HTTP
      port: 80
      allowedRoutes:
        namespaces:
          from: All
EOF

Nach dem Einrichten der benötigten Ressourcen, für das Gateway API, kontrollieren wir ob `GatewayClass` und `Gateways` auf "True" stehen. Dann war die Konfiguration erfolgreich.

In [ ]:
%%bash
kubectl get gatewayclass
echo ""
kubectl -n projectcontour get gateways
echo ""
kubectl -n projectcontour get all

- - -

## Microservices

Das Beispiel besteht aus drei Microservices: **Order**, **Customer** und **Catalog**. 

**Order** nutzt **Catalog** und **Customer** mit der REST-Schnittstelle. Ausserdem bietet jeder Microservice einige HTML-Seiten an.

Statt des ersten Microservices **Webshop**, der als Reverse Proxy konfiguriert ist, wird das Kubernetes Gateway API verwendet.

- - -

Wir Starten die Microservices wie zuvor. Dabei ist zu Beachten, dass die Persona **"Application Developers"** für die Services zuständig sind.

In [ ]:
%%bash
kubectl create namespace ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/catalog-pod.yaml -n ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/customer-pod.yaml -n ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/order-pod.yaml -n ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml -n ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml -n ms-gateway
kubectl apply -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml -n ms-gateway

Und als letzten Schritt die Persona **"Application Developer"** die HTTP Routen

**ACHTUNG**: läuft diese Notebook nicht in einer Hyper-V Umgebung, muss der Eintrag `hostname` angepasst werden.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: ms-gateway-order
  namespace: ms-gateway  
spec:
  parentRefs:
  - group: gateway.networking.k8s.io
    kind: Gateway
    name: contour
    namespace: projectcontour
  hostnames:
  - "$(hostname).mshome.net"
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /order
    backendRefs:
    - name: order
      port: 8080
%EOF%

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: ms-gateway-customer
  namespace: ms-gateway  
spec:
  parentRefs:
  - group: gateway.networking.k8s.io
    kind: Gateway
    name: contour
    namespace: projectcontour
  hostnames:
  - "$(hostname).mshome.net"
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /customer
    backendRefs:
    - name: customer
      port: 8080
%EOF%

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: ms-gateway-catalog
  namespace: ms-gateway  
spec:
  parentRefs:
  - group: gateway.networking.k8s.io
    kind: Gateway
    name: contour
    namespace: projectcontour
  hostnames:
  - "$(hostname).mshome.net"
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /catalog
    backendRefs:
    - name: catalog
      port: 8080
%EOF%

Überprüfen der erstellen Ressourcen

In [ ]:
! kubectl get pod,services,gatewayclass,gateway,httproute -n ms-gateway

Alle Microservices sind jetzt mittels gleichem DNS erreichbar.

In [ ]:
%%bash
export PORT=$(kubectl get service --namespace projectcontour envoy-contour -o=jsonpath='{ .spec.ports[0].nodePort }')
export SERVER=http://$(hostname).mshome.net:${PORT}
echo "Kunden      : ${SERVER}/customer"
echo "Produkte    : ${SERVER}/catalog"
echo "Bestellungen: ${SERVER}/order"

- - -

Aufräumen

In [ ]:
! kubectl delete namespace ms-gateway